In [1]:
import sqlite3
import pandas as pd
from soynlp.noun import LRNounExtractor_v2
from collections import defaultdict
import re
import numpy as np
import math
from string import punctuation
from soynlp.word import WordExtractor
from soynlp.utils import DoublespaceLineCorpus

# Stopwords 처리
pattern1 = re.compile(r'[{}]'.format(re.escape(punctuation))) # punctuation 제거
pattern2 = re.compile(r'[^가-힣 ]') # 특수문자, 모음, 숫자, 영어 제거
pattern3 = re.compile(r'\s{2,}') # white space 1개로 바꾸기.

class Extracter:
    def __init__(self, df):
        self.df = df
        self.noun_extractor = LRNounExtractor_v2(verbose=True)
        self.word_extractor = WordExtractor(min_frequency=math.floor(len(self.df)*0.0001))
        
    def cleaning(self):
        self.df['head'] = self.df['head'].map(lambda x:pattern3.sub(' ',
                                                        pattern2.sub('',
                                                         pattern1.sub('', x))))
        return self.df

    def extract_nouns(self):
        tempary = np.linspace(0,1,11)
        nouns = [self.noun_extractor.train_extract(_['head'], min_noun_frequency=math.floor(len(self.df)*0.0001)) for _ in [self.df.iloc[math.ceil(len(self.df)*tempary[_]):math.ceil(len(self.df)*tempary[_+1])] for _ in range(len(tempary)-1)]]
        words = {k:v for i in range(len(nouns)) for k,v in nouns[i].items() if len(k) > 1}
        return words

    def search_dict(self,nouns):
        # 사전 검색 결과 없는 단어 추출
        conn = sqlite3.connect('kr_korean.db')
        cur = conn.cursor()
        data = pd.read_sql('SELECT word FROM kr', conn)
        data = ' '.join(data['word'])
        return pd.DataFrame([_ for _ in nouns if _[0] not in data])
    
    # 의미 추출을 위한 training data set 생성
    def extract_sent(self, words):
        sent = defaultdict(lambda:0)
        for w in new_words[0]:
            temp = [s for s in df['head'] if w in s]
            sent[w] = '  '.join(temp)
        return sent
            
    def extract_statistic_value(self, corpus):
#         sent = [s for w in new_words[0] for s in self.df['head'] if w in s]
#         sent = pd.DataFrame(sent)
#         sent.drop_duplicates(keep='first', inplace=True)
        self.word_extractor.train(corpus)
        return self.word_extractor.extract()

In [2]:
conn = sqlite3.connect('Humor.db')
cur = conn.cursor()
df = pd.read_sql('SELECT head FROM head',conn)
df.drop_duplicates(keep='first')
ext = Extracter(df)
df = ext.cleaning()

new_words = ext.search_dict(sorted(ext.extract_nouns().items(),key=lambda _:_[1], reverse=True))
sent = ext.extract_sent(new_words)

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 141099 from 125252 sents. mem=0.485 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=532046, mem=0.680 Gb
[Noun Extractor] batch prediction was completed for 37314 words
[Noun Extractor] checked compounds. discovered 21766 compounds
[Noun Extractor] postprocessing detaching_features : 419 -> 418
[Noun Extractor] postprocessing ignore_features : 418 -> 392
[Noun Extractor] postprocessing ignore_NJ : 392 -> 389
[Noun Extractor] 389 nouns (21766 compounds) with min frequency=125
[Noun Extractor] flushing was done. mem=0.747 Gb                    
[Noun Extractor] 30.43 % eojeols are covered
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 150363 from 125252 sents. mem=0.802 Gb                    
[Noun Extractor] complete eojeol counter -> lr grap

In [12]:
static = defaultdict(lambda:0)
for k,v in sent.items():
    temp = ext.extract_statistic_value(v)
    break

training was done. used memory 0.872 Gb
all cohesion probabilities was computed. # words = 0
all branching entropies was computed # words = 0
all accessor variety was computed # words = 0
